In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

In [2]:
# Hyperparameters

RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

In [3]:
def steel_exec(epoch_count=10, mb_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [4]:
def load_steel_dataset():
    with open('../../DataSets/steel_faults.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

In [5]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [6]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}]'.format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [7]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_dix, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [8]:
def run_train(x,y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x,y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [9]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [10]:
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy) # d L/d entropy의 기울기
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [11]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [12]:
def softmax(x):
    max_elem = np.max(x, axis=1) # 최대항 감산을 위한 최대항 추출
    diff = (x.transpose() - max_elem).transpose() # 최대항 감산
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y): # 안쓰임
    mb_size, nom_size = x.shape # mb: minbatch
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n,i,j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n, i]
    return derve

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1) # 1.0e-10 : log 폭주 방지용

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [13]:
steel_exec()

Epoch 1: loss=20.337, accuracy=0.117/0.136]
Epoch 2: loss=20.278, accuracy=0.119/0.136]
Epoch 3: loss=20.278, accuracy=0.119/0.136]
Epoch 4: loss=18.406, accuracy=0.201/0.207]
Epoch 5: loss=18.257, accuracy=0.207/0.207]
Epoch 6: loss=18.257, accuracy=0.207/0.207]
Epoch 7: loss=18.257, accuracy=0.207/0.207]
Epoch 8: loss=18.257, accuracy=0.207/0.207]
Epoch 9: loss=18.257, accuracy=0.207/0.207]
Epoch 10: loss=18.257, accuracy=0.207/0.207]

Final Test: final accuracy = 0.207


In [14]:
print(weight)
print(bias)

[[ 4.56774713e+54 -7.68343676e+65  5.07646040e+54  5.87332594e+54
  -5.04583906e+65  1.27292758e+66  4.97436200e+55]
 [ 4.41477741e+54 -8.56809685e+65  1.24937133e+55  5.90534286e+54
  -5.09498685e+65  1.36630837e+66  5.22302674e+55]
 [ 4.66394730e+57 -1.99026159e+69  1.00256443e+59  5.25177082e+57
  -1.78738855e+69  3.77765015e+69  9.73777733e+58]
 [ 4.66390105e+57 -1.99046392e+69  1.00261071e+59  5.25179928e+57
  -1.78740576e+69  3.77786967e+69  9.73789722e+58]
 [-2.36569458e+54 -6.44245523e+66  3.73182095e+56  8.53784510e+52
  -6.71686369e+64  6.50962387e+66 -7.21447911e+54]
 [-4.66024045e+53 -5.28338668e+65  1.86125023e+55  3.91317901e+52
  -1.14678161e+64  5.39806484e+65  1.77516029e+54]
 [-9.96081929e+52 -3.80076189e+65  1.06438469e+55  2.84594837e+52
  -1.72017241e+64  3.97277913e+65  1.16683883e+54]
 [-1.63033710e+56 -7.76865901e+68  4.13246004e+58  1.00995593e+55
  -7.85053922e+66  7.84716440e+68 -1.07593213e+57]
 [ 5.54959932e+53 -1.55634646e+65  2.41905611e+54  7.47061447e+5

In [16]:
LEARNING_RATE = 0.001
steel_exec()

Epoch 1: loss=18.396, accuracy=0.199/0.199]
Epoch 2: loss=18.911, accuracy=0.179/0.199]
Epoch 3: loss=18.718, accuracy=0.187/0.182]
Epoch 4: loss=18.272, accuracy=0.206/0.182]
Epoch 5: loss=20.188, accuracy=0.123/0.095]
Epoch 6: loss=20.619, accuracy=0.105/0.095]
Epoch 7: loss=20.619, accuracy=0.105/0.095]
Epoch 8: loss=20.619, accuracy=0.105/0.095]
Epoch 9: loss=20.619, accuracy=0.105/0.095]
Epoch 10: loss=20.619, accuracy=0.105/0.095]

Final Test: final accuracy = 0.095
